<a href="https://colab.research.google.com/github/m-hassaan-ar/ai-lab/blob/main/lab6/q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# task4_multi_robot_delivery.py
!pip install ortools
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def create_data():
    data = {}
    # 6×6 grid flattened to 36 locations
    data['locations'] = [(i,j) for i in range(6) for j in range(6)]
    n = len(data['locations'])
    # distance = Manhattan
    data['dist_matrix'] = [
        [
            abs(x1-x2) + abs(y1-y2)
            for (x2,y2) in data['locations']
        ]
        for (x1,y1) in data['locations']
    ]
    data['num_robots'] = 5
    data['depot'] = 0  # start at location (0,0)
    # pretend 10 packages are at locations 1..10 with weight demands
    data['demands'] = [0]*n
    weights = [2,4,6,8,3,5,7,9,1,10]
    for i, w in enumerate(weights, start=1):
        data['demands'][i] = w
    # each robot capacity
    data['capacities'] = [15]*data['num_robots']
    return data

def main():
    data = create_data()
    manager = pywrapcp.RoutingIndexManager(
        len(data['dist_matrix']),
        data['num_robots'],
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)

    def dist_cb(fr, to):
        return data['dist_matrix'][manager.IndexToNode(fr)][manager.IndexToNode(to)]
    routing.SetArcCostEvaluatorOfAllVehicles(routing.RegisterTransitCallback(dist_cb))

    def demand_cb(idx):
        return data['demands'][manager.IndexToNode(idx)]
    capacity_cb = routing.RegisterUnaryTransitCallback(demand_cb)
    routing.AddDimensionWithVehicleCapacity(
        capacity_cb,
        0,            # no slack
        data['capacities'],
        True,         # start with zero load
        "Capacity"
    )

    search_params = pywrapcp.DefaultRoutingSearchParameters()
    search_params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    sol = routing.SolveWithParameters(search_params)
    if sol:
        capacity_dim = routing.GetDimensionOrDie("Capacity")
        for v in range(data['num_robots']):
            idx = routing.Start(v)
            route = []
            load = 0
            while not routing.IsEnd(idx):
                node = manager.IndexToNode(idx)
                load = sol.Value(capacity_dim.CumulVar(idx))
                route.append(node)
                idx = sol.Value(routing.NextVar(idx))
            route.append(manager.IndexToNode(idx))
            print(f"Robot {v} route: {route}  Load: {load}")

if __name__ == "__main__":
    main()


Robot 0 route: [0, 0]  Load: 0
Robot 1 route: [0, 1, 10, 11, 5, 0]  Load: 12
Robot 2 route: [0, 9, 3, 4, 0]  Load: 7
Robot 3 route: [0, 2, 8, 0]  Load: 4
Robot 4 route: [0, 6, 12, 18, 24, 30, 31, 32, 33, 34, 35, 29, 28, 27, 26, 25, 19, 20, 21, 22, 23, 17, 16, 15, 14, 13, 7, 0]  Load: 5
